In [ ]:
import os
import random
import shutil
import subprocess
import urllib.parse
from datetime import datetime
import requests

# === INPUT ===
original_url = input("Enter the original signed URL: ").strip()
filename = input("Enter the desired filename (without extension): ").strip()
output_mp4 = f"{filename}.mp4"
output_mp3 = f"{filename}.mp3"

# === Validate domain ===
if not original_url.startswith("https://web.sancdn.com/"):
    raise ValueError("Only URLs from web.sancdn.com are allowed.")

print("\nOriginal URL:")
print(original_url)

# === Parse URL ===
parsed_url = urllib.parse.urlparse(original_url)
query_params = urllib.parse.parse_qs(parsed_url.query)
base_path = f"{parsed_url.scheme}://{parsed_url.netloc}{parsed_url.path}"

expires = query_params.get("Expires", [None])[0]
signature = query_params.get("Signature", [None])[0]
key_pair_id = query_params.get("Key-Pair-Id", [None])[0]

if not all([expires, signature, key_pair_id]):
    raise ValueError("Missing one or more required query parameters: Expires, Signature, Key-Pair-Id")

try:
    expires_int = int(expires)
except ValueError:
    raise ValueError("Could not parse 'Expires' as integer.")

# === Adjust Expires ===
offset = random.randint(600, 900)
new_expires = expires_int + offset
readable_expiry = datetime.utcfromtimestamp(new_expires).astimezone().strftime('%Y-%m-%d %H:%M:%S %Z')
print(f"\nNew Expires timestamp: {new_expires} ({readable_expiry})")

# === Construct modified URL ===
# NOTE: Signature and Key-Pair-Id are not re-signed for the new expires value. This may cause download failure.
modified_url = f"{base_path}?Expires={new_expires}&Signature={urllib.parse.quote(signature)}&Key-Pair-Id={urllib.parse.quote(key_pair_id)}"
print("\nModified URL:")
print(modified_url)

# === Download video ===
headers = {
    "User-Agent": "Mozilla/5.0",
    "Referer": "https://www.san.com/"
}

print("\nDownloading video...")
try:
    with requests.get(modified_url, headers=headers, stream=True) as r:
        r.raise_for_status()
        with open(output_mp4, 'wb') as f:
            for chunk in r.iter_content(chunk_size=8192):
                f.write(chunk)
except Exception as e:
    print(f"Download failed: {e}")
    raise SystemExit(1)

# === Convert to MP3 ===
if not shutil.which("ffmpeg"):
    raise EnvironmentError("ffmpeg is not installed or not found in PATH.")

print("\nConverting to MP3...")
ffmpeg_cmd = [
    "ffmpeg",
    "-i", output_mp4,
    "-vn",
    "-acodec", "libmp3lame",
    "-q:a", "2",
    output_mp3
]

try:
    subprocess.run(ffmpeg_cmd, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL, check=True)
    os.remove(output_mp4)
    print(f"\n✅ Done! Your MP3 file is saved as: {output_mp3}")
except Exception as e:
    print(f"Conversion failed: {e}")
    raise